## Delete Rejected CryoSPARC Exposures

This program will walk you through deleting exposures that were rejected from your CryoSPARC *Sort Exposures* job. To run this, you must have a valid CryoSPARC account, project, and *Sort Exposures* job, as well as read/write access to where the files are stored. 

This program ***will delete data!!!1!***. Use it with extreme caution. The author of this program bears no responsibility for accidentally deleted data. 

To use this program, click "Run" at each box, starting at the top. If you are prompted to type something, type it in and then press 'Enter' to proceed. 

To reset this for a new project, on the top menu, selet 'Kernel --> Restart & Clear Output'

### Import necessary Python libraries:

In [ ]:
import os
from pathlib import Path 
from getpass import getpass
from cryosparc.tools import CryoSPARC

### Setup CryoSPARC instance & account:

In [ ]:
cs_license = input("Enter your CryoSPARC license: ")

In [ ]:
cs_hostname = input("Enter your CryoSPARC web URL: ")

In [ ]:
cs_email = input("Enter your CryoSPARC account email: ")

In [ ]:
cs_password = getpass("Enter your CryoSPARC account password: ")

In [ ]:
cs = CryoSPARC(license=cs_license, email=cs_email, password=cs_password, host=cs_hostname, base_port=39000)

### Get CryoSPARC Project and Job Numbers:

In [ ]:
project = cs.find_project(str('P' + str(input("Which cryoSPARC project number? "))))

In [ ]:
job = project.find_job(str('J' + str(input("Which cryoSPARC job number contains the rejected exposures? "))))

### Get rejected exposures:

In [ ]:
exposures_rejected = job.load_output('exposures_rejected')
project_dir = Path(project.dir())
movies_list = []

for group in ('movie_blob',):
    field = group + "/path"
    if field in exposures_rejected:
        print(f"Found {len(exposures_rejected)} rejected exposures. Please wait, this may take some time...")
        for rel_path in exposures_rejected['movie_blob/path']:
            abs_path = project_dir / rel_path
            resolved_path=Path.resolve(abs_path)
            movies_list.append(str(resolved_path))
            
print("Done.")

### Save list of filenames: 
Run this if you want to save a list of filenames to be deleted. Otherwise, skip down to **Delete**

In [ ]:
with open("rejected_exposures.txt", "a") as output_file:
    print(str(movies_list).replace(',', '\n'), file=output_file)
output_file.close()

### Delete rejected exposures:

***WARNING*** THIS WILL DELETE ALL THE MOVIES THAT WERE REJECTED IN THE *Sort Exposures* JOB !!!!!!!

This will **not** delete particle locations, extracted particles, or even aligned micrographs. This will only delete the **movies** associated with rejected exposures in the CryoSPARC *Sort Exposures* job you specified above.

In [ ]:
print("Beginning file removal, please wait, this may take some time...")
for i in range(len(movies_list)):
    if Path(movies_list[i]).exists == True:
        os.remove(movies_list[i])
    else:
        print("File", str(movies_list[i]), "could not be found")
print("Done.")